In [6]:
import datetime
from sklearn_utils.utils import SkUtilsIO,filter_by_label
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import numpy as np
import pandas as pd
from sklearn import preprocessing
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

In [7]:
X, y = SkUtilsIO('BC.csv').from_csv(
    label_column='stage')
y = ['healthy' if i == 'h' else 'bc' for i in y]

In [8]:
new_y = []
for item in y:
    if item == "bc":
        new_y.append(1)
    else:
        new_y.append(0)
np_y_new = np.array(new_y)

In [9]:
X_df = pd.read_excel('new_analysis.xlsx')

In [10]:
X_train, X_test, y_train, y_test = train_test_split(np.nan_to_num(X_df), y, test_size=0.1, random_state=42)

Logistic Regression

In [11]:
c_space = np.logspace(-5, 8, 15)
param_grid = {'C': c_space}
logreg = LogisticRegression(C=0.3e-6)
logreg_cv = GridSearchCV(logreg, param_grid, cv=5)
logreg_cv.fit(X_train,y_train)
# logreg.fit(X_train, y_train)
y_pred = logreg_cv.predict(X_test)
kf = StratifiedKFold(n_splits=10)
# scores = cross_val_score(logreg, np.nan_to_num(X_df), np_y_new, cv=kf, scoring='f1')
log_scores = cross_val_score(logreg_cv, np.nan_to_num(X_df), np_y_new, cv=kf, scoring='f1_micro')
print('kfold test: %s' % log_scores)
print('mean: %s' % log_scores.mean().round(3))
print('std: %s' % log_scores.std().round(3))

kfold test: [0.77272727 0.77272727 0.72727273 0.81818182 0.95454545 0.77272727
 0.95238095 0.85714286 0.85       0.95      ]
mean: 0.843
std: 0.081


Random Forest

In [12]:
rf = RandomForestClassifier(max_depth=4, random_state=0)
rf.fit(X_train, y_train)
kf = StratifiedKFold(n_splits=10)
rf_scores = cross_val_score(rf, np.nan_to_num(X_df), np_y_new, cv=kf, scoring='f1_micro')
print('kfold test: %s' % rf_scores)
print('mean: %s' % rf_scores.mean().round(3))
print('std: %s' % rf_scores.std().round(3))

kfold test: [0.77272727 0.72727273 0.77272727 0.81818182 0.77272727 0.77272727
 0.9047619  0.85714286 0.85       0.9       ]
mean: 0.815
std: 0.058


SVM

In [13]:
svm = SVC(gamma='auto')
svm.fit(X_train, y_train)
kf = StratifiedKFold(n_splits=10)
svm_scores = cross_val_score(svm, np.nan_to_num(X_df), np_y_new, cv=kf, scoring='f1_micro')
print('kfold test: %s' % svm_scores)
print('mean: %s' % svm_scores.mean().round(3))
print('std: %s' % svm_scores.std().round(3))

kfold test: [0.63636364 0.63636364 0.63636364 0.63636364 0.63636364 0.63636364
 0.66666667 0.66666667 0.65       0.65      ]
mean: 0.645
std: 0.012


In [14]:
kf = StratifiedKFold(n_splits=10, random_state=43)
logreg = LogisticRegression(C=0.3e-6)
logreg_cv_split = GridSearchCV(logreg, param_grid, cv=5)
logreg_results = []
for train_index, test_index in kf.split(np.nan_to_num(X_df), np_y_new):
        X_train, y_train= np.nan_to_num(X_df)[train_index], np_y_new[train_index]
        X_test, y_test= np.nan_to_num(X_df)[test_index], np_y_new[test_index]
        
        clf = logreg_cv_split.fit(X_train, y_train)
        # print(classification_report(clf.predict(X_test), y_test))
        logreg_results.append(classification_report(clf.predict(X_test), y_test))

In [15]:
rf_results = []
kf = StratifiedKFold(n_splits=10, random_state=43)
randomforest_split = RandomForestClassifier(max_depth=4, random_state=0)
for train_index, test_index in kf.split(np.nan_to_num(X_df), np_y_new):
        X_train, y_train= np.nan_to_num(X_df)[train_index], np_y_new[train_index]
        X_test, y_test= np.nan_to_num(X_df)[test_index], np_y_new[test_index]
        
        clf = randomforest_split.fit(X_train, y_train)
        # print(classification_report(clf.predict(X_test), y_test))
        rf_results.append(classification_report(clf.predict(X_test), y_test))

In [16]:
svm_results = []
kf = StratifiedKFold(n_splits=10, random_state=43)
svm_split = SVC(gamma='auto')
for train_index, test_index in kf.split(np.nan_to_num(X_df), np_y_new):
        X_train, y_train= np.nan_to_num(X_df)[train_index], np_y_new[train_index]
        X_test, y_test= np.nan_to_num(X_df)[test_index], np_y_new[test_index]
        
        clf = svm_split.fit(X_train, y_train)
        # print(classification_report(clf.predict(X_test), y_test))
        svm_results.append(classification_report(clf.predict(X_test), y_test))

C:\Users\alperdokay\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\alperdokay\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\alperdokay\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\alperdokay\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no t

# Logistic Regression Fold Based Scores

In [17]:
for item in logreg_results:
    print(item)

              precision    recall  f1-score   support

           0       0.50      0.80      0.62         5
           1       0.93      0.76      0.84        17

    accuracy                           0.77        22
   macro avg       0.71      0.78      0.73        22
weighted avg       0.83      0.77      0.79        22

              precision    recall  f1-score   support

           0       0.38      1.00      0.55         3
           1       1.00      0.74      0.85        19

    accuracy                           0.77        22
   macro avg       0.69      0.87      0.70        22
weighted avg       0.91      0.77      0.81        22

              precision    recall  f1-score   support

           0       0.25      1.00      0.40         2
           1       1.00      0.70      0.82        20

    accuracy                           0.73        22
   macro avg       0.62      0.85      0.61        22
weighted avg       0.93      0.73      0.79        22

              preci

# Random Forest Fold Based Scores

In [18]:
for item in rf_results:
    print(item)

              precision    recall  f1-score   support

           0       0.50      0.80      0.62         5
           1       0.93      0.76      0.84        17

    accuracy                           0.77        22
   macro avg       0.71      0.78      0.73        22
weighted avg       0.83      0.77      0.79        22

              precision    recall  f1-score   support

           0       0.25      1.00      0.40         2
           1       1.00      0.70      0.82        20

    accuracy                           0.73        22
   macro avg       0.62      0.85      0.61        22
weighted avg       0.93      0.73      0.79        22

              precision    recall  f1-score   support

           0       0.38      1.00      0.55         3
           1       1.00      0.74      0.85        19

    accuracy                           0.77        22
   macro avg       0.69      0.87      0.70        22
weighted avg       0.91      0.77      0.81        22

              preci

# SVM Fold Based Scores

In [19]:
for item in svm_results:
    print(item)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.64      0.78        22

    accuracy                           0.64        22
   macro avg       0.50      0.32      0.39        22
weighted avg       1.00      0.64      0.78        22

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.64      0.78        22

    accuracy                           0.64        22
   macro avg       0.50      0.32      0.39        22
weighted avg       1.00      0.64      0.78        22

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.64      0.78        22

    accuracy                           0.64        22
   macro avg       0.50      0.32      0.39        22
weighted avg       1.00      0.64      0.78        22

              preci

# GENERAL OVERVIEW

In [20]:
df = pd.DataFrame({'classifiers': ['logreg', 'rf', 'svm'], 'mean': [log_scores.mean().round(3), rf_scores.mean().round(3),svm_scores.mean().round(3)]})
ax = df.plot.bar(x='classifiers', y='mean', rot=0)
df.sort_values('mean', ascending=False)
# df.to_excel('new_test.xlsx')

,classifiers,mean
0,logreg,0.843
1,rf,0.815
2,svm,0.645


In [23]:
from statsmodels.sandbox.stats.multicomp import multipletests
from sklearn.feature_selection import VarianceThreshold, f_classif


def variance_threshold_on_df(df: pd.DataFrame, threshold=0):
    vt = VarianceThreshold(threshold)
    vt.fit(df.values)
    return df.iloc[:, vt.variances_ > threshold]

def feature_importance_anova(X,
                             y,
                             threshold=0.001,
                             correcting_multiple_hypotesis=True,
                             method='fdr_bh',
                             alpha=0.1,
                             sort_by='F'):
    '''
    Provide signifance for features in dataset with anova using multiple hypostesis testing
    :X: List of dict with key as feature names and values as features
    :y: Labels
    :threshold: Low-variens threshold to eliminate low varience features
    :correcting_multiple_hypotesis: corrects p-val with multiple hypotesis testing
    :method: method of multiple hypotesis testing
    :alpha: alpha of multiple hypotesis testing
    :sort_by: sorts output dataframe by pval or F
    :return: DataFrame with F and pval for each feature with their average values 
    '''
    df = variance_threshold_on_df(
        pd.DataFrame.from_records(X), threshold=threshold)

    F, pvals = f_classif(df.values, y)

    if correcting_multiple_hypotesis:
        _, pvals, _, _ = multipletests(pvals, alpha=alpha, method=method)

    df['labels'] = y
    df_mean = df.groupby('labels').mean().T

    df_mean['F'] = F
    df_mean['pval'] = pvals

    return df_mean.sort_values(sort_by, ascending=True)

In [24]:
df_pathways = feature_importance_anova(X_df.fillna(0), y)
df_pathways.head(10)

labels,bc,healthy,F,pval
Histidine metabolism,0.101254,9.611792e-11,0.000037,0.995139
Fatty acid synthesis,-1.107004,1.545849e-10,0.013538,0.920087
Tryptophan metabolism,-0.493611,4.192043e-11,0.026944,0.894274
Oxidative phosphorylation,-3.349415,2.385875e-10,0.049346,0.859751
Propanoate metabolism,-3.611316,1.181632e-10,0.098604,0.797520
Pentose phosphate pathway,-1.919117,1.988922e-10,0.120906,0.781957
Alanine and aspartate metabolism,7.641108,1.219082e-10,0.121783,0.781957
beta-Alanine metabolism,-3.833332,1.703954e-10,0.147693,0.775498
Vitamin B2 metabolism,3.186711,8.234667e-11,0.151478,0.775498
Glutathione metabolism,-6.607954,7.295240e-11,0.211729,0.736715
